In [1]:
import os
import os
import numpy as np 
import scipy as sp 
import matplotlib as mpl 
import matplotlib.cm as cm 
import matplotlib.pyplot as plt 
import pandas as pd 
import nltk
import re
import csv
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import SGDClassifier
from scipy.sparse import hstack
from sklearn.metrics import confusion_matrix
from time import time

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

%matplotlib inline

C:\Program Files\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
class_dataset = pd.read_csv('REDW_Raw_sa.csv')

In [3]:
class_dataset.shape

(249196, 6)

In [4]:
class_dataset.head()

,Unnamed: 0,Date,Comments,class,Timesatmp,date1
0,1,Wed Mar 22 11:26:19 2017_1837642,Thread 5 advanced to log sequence 28271 (LGWR ...,LGWR switch,Wed Mar 22 11:26:19 2017,2017-03-22 11:26:19
1,2,Wed Sep 28 08:34:45 2016_704018,Archived Log entry 80694 added for thread 5 se...,Archived_thread_error,Wed Sep 28 08:34:45 2016,2016-09-28 08:34:45
2,3,Mon Jul 11 14:01:48 2016_246776,diag_adl:Some indexes or index [sub]partitions...,Diagonal Adl error,Mon Jul 11 14:01:48 2016,2016-07-11 14:01:48
3,4,Mon Apr 17 09:14:58 2017_2040135,**********************************************...,Tns Error,Mon Apr 17 09:14:58 2017,2017-04-17 09:14:58
4,5,Thu Sep 29 15:20:17 2016_719382,**********************************************...,Tns Error,Thu Sep 29 15:20:17 2016,2016-09-29 15:20:17


In [7]:
stop = set(stopwords.words('english'))

In [8]:
doc_complete = class_dataset['Comments'].tolist()

In [9]:
class_dataset.shape

(249196, 6)

In [10]:
len(doc_complete)

249196

In [51]:
def clean(doc):
    doc = " ".join([i.replace('*', '') for i in doc.lower().split()])
    doc = " ".join([i.replace(':', ' ') for i in doc.split()])
    doc = " ".join([i.replace('.', ' ') for i in doc.split()])
    doc = " ".join([i.replace('=', '') for i in doc.split()])
    doc = " ".join([i.replace('/', ' ') for i in doc.split()])
    doc = " ".join([i.replace(')', ' ') for i in doc.split()])
    doc = " ".join([i.replace('(', ' ') for i in doc.split()])
    doc = " ".join([i.replace('"', ' ') for i in doc.split()])
    doc = " ".join([i.replace('-', ' ') for i in doc.split()])
    doc = " ".join([i.replace('_', ' ') for i in doc.split()])
    doc = " ".join([i for i in doc.split() if not i.isdigit()])
    doc = " ".join([i for i in doc.split() if i.isalpha()])
    doc = " ".join([i for i in doc.split() if i not in stop])
    return doc

In [53]:
doc_clean = [clean(doc).split() for doc in doc_complete]

In [54]:
len(doc_clean)

249196

In [55]:
import gensim
from gensim import corpora

C:\Program Files\Anaconda2\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [56]:
dictionary = corpora.Dictionary(doc_clean)

In [57]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [58]:
Lda = gensim.models.ldamodel.LdaModel

In [61]:
ldamodel = Lda(doc_term_matrix, num_topics=15, id2word = dictionary, passes=50)

In [62]:
print(ldamodel.print_topics(num_topics=15, num_words=3))

[(0, u'0.086*"process" + 0.058*"background" + 0.052*"starting"'), (1, u'0.155*"data" + 0.129*"note" + 0.125*"disk"'), (2, u'0.147*"slave" + 0.139*"ksfv" + 0.138*"exiting"'), (3, u'0.082*"table" + 0.080*"index" + 0.080*"marked"'), (4, u'0.107*"ora" + 0.039*"file" + 0.039*"line"'), (5, u'0.083*"error" + 0.069*"connect" + 0.056*"fatal"'), (6, u'0.210*"size" + 0.109*"new" + 0.107*"completed"'), (7, u'0.044*"data" + 0.030*"server" + 0.029*"check"'), (8, u'0.489*"diag" + 0.486*"adl" + 0.004*"tns"'), (9, u'0.091*"table" + 0.085*"marked" + 0.085*"index"'), (10, u'0.126*"id" + 0.126*"log" + 0.124*"dest"'), (11, u'0.123*"group" + 0.121*"tp" + 0.121*"onlinelog"'), (12, u'0.418*"os" + 0.194*"running" + 0.192*"slave"'), (13, u'0.079*"code" + 0.079*"nt" + 0.079*"err"'), (14, u'0.085*"inbound" + 0.082*"ora" + 0.081*"warning"')]


In [64]:
from time import time

In [67]:
t0 = time()
ldamodel = Lda(doc_term_matrix, num_topics=15, id2word = dictionary, iterations=5000)
print 'Time taken:', time() - t0, 'seconds'

Time taken: 249.196000099 seconds


In [68]:
print(ldamodel.print_topics(num_topics=15, num_words=3))

[(0, u'0.070*"process" + 0.061*"background" + 0.055*"starting"'), (1, u'0.067*"table" + 0.066*"marked" + 0.066*"index"'), (2, u'0.079*"nt" + 0.079*"err" + 0.079*"code"'), (3, u'0.273*"size" + 0.132*"completed" + 0.131*"new"'), (4, u'0.070*"roles" + 0.052*"name" + 0.043*"maximum"'), (5, u'0.193*"data" + 0.153*"note" + 0.144*"disk"'), (6, u'0.135*"group" + 0.135*"tp" + 0.133*"onlinelog"'), (7, u'0.232*"immediate" + 0.231*"kill" + 0.127*"session"'), (8, u'0.079*"os" + 0.078*"slave" + 0.077*"running"'), (9, u'0.125*"added" + 0.122*"id" + 0.120*"thread"'), (10, u'0.094*"nfs" + 0.059*"check" + 0.057*"specified"'), (11, u'0.083*"diag" + 0.081*"table" + 0.080*"adl"'), (12, u'0.089*"ora" + 0.034*"tp" + 0.031*"file"'), (13, u'0.427*"adl" + 0.427*"diag" + 0.012*"tns"'), (14, u'0.131*"ora" + 0.041*"line" + 0.039*"trace"')]


In [69]:
import numpy as np

In [ ]:
df["clean"] = spa